In [1]:
import numpy as np
import random

def get_distance_matrix():
    while True:
        try:
            num_cities = int(input("Enter the number of cities: "))
            print(f"Enter the {num_cities}x{num_cities} distance matrix (row by row, space-separated):")
            distance_matrix = []
            for i in range(num_cities):
                while True:
                    row = input(f"Row {i+1}: ").strip().split()
                    if len(row) != num_cities:
                        print(f"Each row must have exactly {num_cities} values. Try again.")
                        continue
                    try:
                        row = list(map(float, row))
                        distance_matrix.append(row)
                        break
                    except ValueError:
                        print("Invalid input. Please enter numeric values.")
            return np.array(distance_matrix)
        except ValueError:
            print("Invalid number of cities. Please enter an integer.")

def ant_colony_optimization(distance_matrix, num_ants=10, num_iterations=50, evaporation_rate=0.5, pheromone_constant=1.0, heuristic_constant=1.0):
    num_cities = len(distance_matrix)
    pheromone = np.ones((num_cities, num_cities))
    visibility = np.where(distance_matrix > 0, 1 / distance_matrix, 0)

    best_route = None
    shortest_distance = float('inf')
    
    for _ in range(num_iterations):
        ant_routes = []
        for _ in range(num_ants):
            current_city = random.randint(0, num_cities - 1)
            visited_cities = [current_city]
            route = [current_city]
            
            while len(visited_cities) < num_cities:
                probabilities = []
                total = 0
                for city in range(num_cities):
                    if city not in visited_cities:
                        pheromone_value = pheromone[current_city][city]
                        visibility_value = visibility[current_city][city]
                        prob = (pheromone_value ** pheromone_constant) * (visibility_value ** heuristic_constant)
                        probabilities.append((city, prob))
                        total += prob
                
                if not probabilities or total == 0:
                    break  # No move possible
                
                cities, probs = zip(*probabilities)
                normalized_probs = [p / total for p in probs]
                selected_city = random.choices(cities, weights=normalized_probs)[0]

                route.append(selected_city)
                visited_cities.append(selected_city)
                current_city = selected_city
            
            ant_routes.append(route)
        
        # Update pheromone levels
        delta_pheromone = np.zeros((num_cities, num_cities))
        for route in ant_routes:
            distance = sum(distance_matrix[route[i]][route[(i + 1) % num_cities]] for i in range(num_cities))
            for i in range(num_cities):
                a, b = route[i], route[(i + 1) % num_cities]
                if distance > 0:
                    delta_pheromone[a][b] += 1 / distance
                    delta_pheromone[b][a] += 1 / distance
        
        pheromone = (1 - evaporation_rate) * pheromone + delta_pheromone
        
        # Find the best route
        for route in ant_routes:
            distance = sum(distance_matrix[route[i]][route[(i + 1) % num_cities]] for i in range(num_cities))
            if distance < shortest_distance:
                shortest_distance = distance
                best_route = route
    
    print("Best route:", best_route)
    print("Shortest distance:", shortest_distance)

# Run program
distance_matrix = get_distance_matrix()
ant_colony_optimization(distance_matrix)

   

Enter the number of cities: 3
Enter the 3x3 distance matrix (row by row, space-separated):
Row 1: 2 4 1
Row 2: 7 3 2
Row 3: 9 8 5
Best route: [0, 1, 2]
Shortest distance: 15.0
